In [0]:
%sql
USE CATALOG lab; USE SCHEMA data;

In [0]:
%sql
CREATE OR REPLACE TABLE campaigns_silver
AS
SELECT
  CAST(campaign_id AS INT) AS campaign_id,
  campaign_name,
  channel,
  TO_DATE(start_date) AS start_date,
  TO_DATE(end_date) AS end_date,
  CAST(budget_usd AS DOUBLE) AS budget_usd,
  objective
FROM lab.data.campaigns;

In [0]:
%sql
CREATE OR REPLACE TABLE daily_spend_silver
AS
SELECT
  CAST(campaign_id AS INT) AS campaign_id,
  TO_DATE(date) AS date,
  CAST(spend_usd AS DOUBLE) AS spend_usd
FROM lab.data.daily_spend
WHERE spend_usd >= 0;

In [0]:
%sql
CREATE OR REPLACE TABLE impressions_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  CAST(campaign_id AS INT) AS campaign_id,
  device,
  country,
  placement,
  TIMESTAMP(event_ts) AS event_ts,
  TO_DATE(event_ts) AS event_date
FROM lab.data.impressions;

In [0]:
%sql
CREATE OR REPLACE TABLE clicks_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  CAST(campaign_id AS INT) AS campaign_id,
  device,
  country,
  placement,
  TIMESTAMP(event_ts) AS event_ts,
  TO_DATE(event_ts) AS event_date
FROM lab.data.clicks;

In [0]:
%sql
CREATE OR REPLACE TABLE customers_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  TO_DATE(signup_date) AS signup_date,
  first_touch_channel
FROM lab.data.customers;

In [0]:
%sql
CREATE OR REPLACE TABLE orders_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  TO_DATE(order_date) AS order_date,
  CAST(revenue_usd AS DOUBLE) AS revenue_usd
FROM lab.data.orders
WHERE revenue_usd >= 0;

In [0]:
%sql
-- Gold: Aggregates and KPIs
CREATE OR REPLACE TABLE daily_channel_metrics_gold
AS
WITH impr AS (
  SELECT event_date AS date, campaign_id, COUNT(*) AS impressions
  FROM lab.data.impressions_silver
  GROUP BY 1,2
), clk AS (
  SELECT event_date AS date, campaign_id, COUNT(*) AS clicks
  FROM lab.data.clicks_silver
  GROUP BY 1,2
), sp AS (
  SELECT date, campaign_id, SUM(spend_usd) AS spend_usd
  FROM lab.data.daily_spend_silver
  GROUP BY 1,2
), ord AS (
  SELECT order_date AS date, user_id, SUM(revenue_usd) AS revenue_usd
  FROM lab.data.orders_silver
  GROUP BY 1,2
), ch AS (
  SELECT campaign_id, channel FROM lab.data.campaigns_silver
)
SELECT
  COALESCE(sp.date, impr.date, clk.date) AS date,
  COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id) AS campaign_id,
  ch.channel,
  COALESCE(sp.spend_usd, 0) AS spend_usd,
  COALESCE(impressions, 0) AS impressions,
  COALESCE(clicks, 0) AS clicks,
  CASE WHEN impressions > 0 THEN clicks / impressions ELSE 0 END AS ctr,
  ord_kpis.revenue_usd,
  CASE WHEN clicks > 0 THEN ord_kpis.orders * 1.0 / clicks ELSE 0 END AS cvr,
  CASE WHEN COALESCE(spend_usd, 0) > 0 THEN COALESCE(ord_kpis.revenue_usd, 0) / spend_usd ELSE 0 END AS roas
FROM sp
FULL OUTER JOIN impr ON sp.date = impr.date AND sp.campaign_id = impr.campaign_id
FULL OUTER JOIN clk ON COALESCE(sp.date, impr.date) = clk.date AND COALESCE(sp.campaign_id, impr.campaign_id) = clk.campaign_id
LEFT JOIN ch ON COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id) = ch.campaign_id
LEFT JOIN (
  SELECT d.date, d.campaign_id,
         COUNT(DISTINCT o.user_id) AS orders,
         SUM(o.revenue_usd) AS revenue_usd
  FROM (
    SELECT DISTINCT event_date AS date, campaign_id, user_id
    FROM lab.data.clicks_silver
  ) d
  LEFT JOIN lab.data.orders_silver o ON o.user_id = d.user_id AND o.order_date = d.date
  GROUP BY 1,2
) ord_kpis ON ord_kpis.date = COALESCE(sp.date, impr.date, clk.date)
          AND ord_kpis.campaign_id = COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id);

In [0]:
%sql
CREATE OR REPLACE TABLE channel_daily_gold
AS
SELECT date, channel,
       SUM(spend_usd) AS spend_usd,
       SUM(impressions) AS impressions,
       SUM(clicks) AS clicks,
       CASE WHEN SUM(impressions) > 0 THEN SUM(clicks) * 1.0 / SUM(impressions) ELSE 0 END AS ctr,
       SUM(revenue_usd) AS revenue_usd,
       CASE WHEN SUM(clicks) > 0 THEN COUNT(*) * 1.0 / SUM(clicks) ELSE 0 END AS cvr_estimate,
       CASE WHEN SUM(spend_usd) > 0 THEN SUM(revenue_usd) / SUM(spend_usd) ELSE 0 END AS roas
FROM lab.data.daily_channel_metrics_gold
GROUP BY 1,2;